We'll be using the 'requests' library for fetching data from the API and 'psycopg2' to connect with our PostgreSQL database. Then need to establish a connection to our PostgreSQL database. We need to define crucial connection details such as the host, database name, username, password, and port number. I  initiate a connection and create a cursor, our tool to execute SQL commands within the database. It is like a virtual pointer moving through our data.

The next step involves setting up our database. We'll create a table named 'Energy' with specific columns like 'timestamp,' 'reportingGroup,' 'locationName,' 'value,' and 'unit.' These columns define the structure of our data. If the 'Energy' table already exists, we drop it and create a new one.

Our table is ready, and it's time to start populating it. We define the values we want to insert – a timestamp, reporting group, location name, value, and unit. This set of information could represent energy consumption data for a specific location at a given time.

I used python Exception handling method to handle potential errors that may occur during the database interaction. It is a good programming practice.  If everything goes smoothly, we close the cursor and the connection.
To fetching data from an external sourcewe, I use the 'requests' library to make an HTTP GET request to a Nuuka Cloud API endpoint. The endpoint contains specific parameters defining the data we want to retrieve, such as the location name, reporting group, and timeframe.

To check if our request was successful, we print the HTTP status code – a code of 200 indicates success in HTTP terms. Additionally, we print the JSON response from the API. JSON provides a structured way of presenting the data we requested.

In [21]:
# Import the 'requests' library for making HTTP requests
import requests

# Install the 'psycopg2' library for connecting to PostgreSQL
! pip install psycopg2

# Import 'psycopg2' for PostgreSQL database interaction
import psycopg2

from datetime import datetime
def convert_to_timestamp(date_str):
    return datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S')


def create_insert_script(timestamp, reportingGroup, locationName, value, unit, cur):
    modified_timestamp = convert_to_timestamp(timestamp)


    insert_data = {
        "timestamp": modified_timestamp,
        "reportingGroup": reportingGroup,
        "locationName": locationName,
        "value": value,
        "unit": unit
    }
    
    insert_query = """
    INSERT INTO Energy ("timestamp", "reportingGroup", "locationName", value, unit)
    VALUES (%s, %s, %s, %s, %s)
    """
    cur.execute(insert_query, (
        insert_data["timestamp"],
        insert_data["reportingGroup"],
        insert_data["locationName"],
        insert_data["value"],
        insert_data["unit"]
    ))



# Create a connection to the PostgreSQL database
conn = psycopg2.connect(dbname="postgres", user="postgres", password="2352", host="localhost")

# Define database connection details
hostname = "localhost"
database = "postgres"
username = "postgres"
pwd = "2352"
port_id = 5432

# Initialize connection and cursor variables
conn = None
cur = None

# Try to establish a connection with exception handling
try:
    conn = psycopg2.connect (
        host=hostname,
        dbname=database,
        user=username,
        password=pwd,
        port=port_id
    )
    
    # Create a cursor for database operations
    cur = conn.cursor()

    # Drop the 'Energy' table if it exists
    cur.execute("DROP TABLE IF EXISTS Energy")

    # Create the 'Energy' table with specific columns and data types
    create_script = """CREATE TABLE IF NOT EXISTS Energy (
                    "timestamp" timestamp without time zone,
                    "reportingGroup" "char",
                    "locationName" character varying COLLATE pg_catalog."default",
                    value real,
                    unit "char"
                )"""
    cur.execute(create_script)
    #conn.commit()

    # Define values to be inserted into the 'Energy' table
    insert_script = "INSERT INTO ENERGY (timestamp, reportingGroup, locationName, value, unit)"
    insert_value = ("2019-01-08T00:00:00", "Electricity", "1000 Hakaniemen kauppahalli", 0.0, "kWh")
    
    
    
    
    
    # Make an HTTP GET request to a Nuuka Cloud API endpoint
    response = requests.get("https://helsinki-openapi.nuuka.cloud/api/v1.0/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=1000%20Hakaniemen%20kauppahalli&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2019-12-31")

    # Print the HTTP status code for testing purposes
    print(response.status_code)

    # Print the JSON response from the API (assuming it's a successful response)
    print(response.json())  # 200 is standard HTTP successful GET response
    data = response.json()
    for entity in data:
        timestamp  = entity["timestamp"] 
        reportingGroup  =entity["reportingGroup"] 
        locationName = entity["locationName"] 
        value  = entity["value"] 
        unit = entity["unit"]
        create_insert_script(timestamp, reportingGroup, locationName, value, unit, cur)
    conn.commit()
except Exception as error: 
    # Print any exception that occurs during the database operations
    print(error)

finally:
    # Close the cursor and connection if they are open
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()



200
[{'timestamp': '2019-01-08T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1000 Hakaniemen kauppahalli', 'value': 0.0, 'unit': 'kWh'}, {'timestamp': '2019-01-09T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1000 Hakaniemen kauppahalli', 'value': 0.0, 'unit': 'kWh'}, {'timestamp': '2019-01-10T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1000 Hakaniemen kauppahalli', 'value': 0.0, 'unit': 'kWh'}, {'timestamp': '2019-01-11T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1000 Hakaniemen kauppahalli', 'value': 20.1, 'unit': 'kWh'}, {'timestamp': '2019-01-12T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1000 Hakaniemen kauppahalli', 'value': 30.560000000000006, 'unit': 'kWh'}, {'timestamp': '2019-01-13T00:00:00', 'reportingGroup': 'Electricity', 'locationName': '1000 Hakaniemen kauppahalli', 'value': 1.3000000000000003, 'unit': 'kWh'}, {'timestamp': '2019-01-14T00:00:00', 'reportingGroup': 'Electricity', 'locationNam